# Hub & Satellite Insights Notebook

Notebook para gerar insights analíticos a partir das tabelas **hub_event** (hub) e **sat_event** (satellite) armazenadas no Postgres (schema `silver`).

## Objetivos
- Carregar dados diretamente do Postgres.
- Perfil inicial das tabelas (tamanho, colunas, tipos, missing).
- Derivar métricas por país e data (contagem de eventos, média de tone e polarity).
- Calcular correlações e rankings.
- Detectar outliers simples (spikes).
- Visualizações (séries temporais, heatmap, scatter de correlação).
- Exportar resultados agregados.

## Assumptions / Suposições
- Tabela `silver.sat_event` contém colunas: `event_hk`, `tone`, `polarity`, `locations`, `date_event` (ou `load_date`).
- Tabela `silver.hub_event` contém ao menos `event_hk` e atributos de ligação. Sem país direto: país inferido via sat_event.
- Coluna `locations` segue padrão GDELT: segmentos separados por `;`, cada segmento com campos separados por `#`, onde posição 2 (index 2) é country code (ISO2 ou ISO3).
- Caso `tone` ou `polarity` inexistam, métricas serão ignoradas ou preenchidas com NaN.

In [15]:
# Seção 1: Imports e Configuração de Conexão (com fallback de host e schema corrigido)
import os, re, math, json, time, socket
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import sqlalchemy as sa
from sqlalchemy.engine import URL

try:
    import pycountry
except ImportError:
    import subprocess, sys as _sys
    subprocess.run([_sys.executable, '-m', 'pip', 'install', 'pycountry'], check=False)
    import pycountry
try:
    import plotly.express as px
    import plotly
except ImportError:
    import subprocess, sys as _sys
    subprocess.run([_sys.executable, '-m', 'pip', 'install', 'plotly'], check=False)
    import plotly.express as px
    import plotly
try:
    import ipywidgets as widgets
except ImportError:
    import subprocess, sys as _sys
    subprocess.run([_sys.executable, '-m', 'pip', 'install', 'ipywidgets'], check=False)
    import ipywidgets as widgets

pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 140)
print('Versões:', 'pandas', pd.__version__, 'plotly', getattr(plotly,'__version__','N/D'))

PG_HOST = os.getenv('PGHOST','postgres')
PG_DB   = os.getenv('PGDATABASE','gdelt')
PG_USER = os.getenv('PGUSER','gdelt_user')
PG_PWD  = os.getenv('PGPASSWORD','gdelt_pass')
PG_PORT = int(os.getenv('PGPORT','5432'))
# Ajuste de schema: dbt materializou em aml_silver
PG_SCHEMA = os.getenv('PG_SCHEMA','aml_silver')

def host_reachable(host:str, port:int, timeout:float=1.5)->bool:
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except Exception:
        return False

if not host_reachable(PG_HOST, PG_PORT):
    if PG_HOST == 'postgres':
        print("Host 'postgres' inacessível do contexto atual. Tentando 'localhost'.")
        PG_HOST = 'localhost'
    else:
        print(f"Host '{PG_HOST}' inacessível. Verifique rede/VPN/port forwarding.")

print(f'Conectando a Postgres host={PG_HOST} db={PG_DB} schema={PG_SCHEMA} user={PG_USER} port={PG_PORT}')

conn_url = URL.create(
    drivername='postgresql+psycopg2',
    username=PG_USER,
    password=PG_PWD,
    host=PG_HOST,
    port=PG_PORT,
    database=PG_DB
)
engine = sa.create_engine(conn_url)

# Descoberta dinâmica opcional caso hub_event não esteja no PG_SCHEMA
try:
    with engine.connect() as conn:
        tables_in_schema = conn.execute(sa.text("SELECT table_name FROM information_schema.tables WHERE table_schema=:sch"), {'sch': PG_SCHEMA}).fetchall()
        if not tables_in_schema:
            print(f'Nenhuma tabela encontrada em {PG_SCHEMA}. Procurando hub_event em outros schemas...')
            found = conn.execute(sa.text("SELECT table_schema FROM information_schema.tables WHERE table_name='hub_event' LIMIT 1")).fetchone()
            if found:
                PG_SCHEMA = found[0]
                print(f'hub_event localizado em schema {PG_SCHEMA}. Atualizando PG_SCHEMA.')
        else:
            print(f'Tabelas no schema {PG_SCHEMA}:', [r[0] for r in tables_in_schema][:6], '...')
except Exception as e:
    print('Falha na verificação de schema:', e)

Versões: pandas 2.2.3 plotly 6.4.0
Host 'postgres' inacessível do contexto atual. Tentando 'localhost'.
Conectando a Postgres host=localhost db=gdelt schema=aml_silver user=gdelt_user port=5432
Tabelas no schema aml_silver: ['hub_event', 'sat_event'] ...
Host 'postgres' inacessível do contexto atual. Tentando 'localhost'.
Conectando a Postgres host=localhost db=gdelt schema=aml_silver user=gdelt_user port=5432
Tabelas no schema aml_silver: ['hub_event', 'sat_event'] ...


In [16]:
# Seção 2: Funções Utilitárias de Carregamento e Helpers
def load_table(table_name: str, limit: int = None):
    full_name = f'{PG_SCHEMA}.{table_name}'
    sql = f'SELECT * FROM {full_name}' + (f' LIMIT {limit}' if limit else '')
    try:
        df = pd.read_sql(sql, engine)
        print(f'Carregado {len(df)} linhas de {full_name}')
        return df
    except Exception as e:
        print('Erro ao carregar', full_name, e)
        return pd.DataFrame()

iso2_to_iso3 = {c.alpha_2: c.alpha_3 for c in pycountry.countries}
def to_iso3(code):
    if not isinstance(code,str): return None
    code = code.upper()
    if len(code)==2 and code in iso2_to_iso3: return iso2_to_iso3[code]
    if len(code)==3: return code
    return None

def extract_country_codes(loc_str):
    if not isinstance(loc_str,str) or loc_str.strip()=='' : return []
    parts = loc_str.split(';')
    out = []
    for p in parts:
        segs = p.split('#')
        if len(segs) >= 3:
            cc = segs[2].upper()
            if re.fullmatch(r'[A-Z]{2,3}', cc): out.append(cc)
    return list(set(out))

def safe_norm(series: pd.Series):
    if series.empty: return series
    mn, mx = series.min(), series.max()
    if mn == mx: return pd.Series([0]*len(series), index=series.index)
    return (series - mn) / (mx - mn)

In [17]:
# Seção 3: Carregar Tabelas hub_event e sat_event
hub_event_df = load_table('hub_event')
sat_event_df = load_table('sat_event')
print('hub_event colunas:', list(hub_event_df.columns))
print('sat_event colunas:', list(sat_event_df.columns))
display(hub_event_df.head())
display(sat_event_df.head())

Carregado 140026 linhas de aml_silver.hub_event
Carregado 140102 linhas de aml_silver.sat_event
hub_event colunas: ['event_hk', 'gkg_id', 'date_id', 'load_date']
sat_event colunas: ['event_hk', 'gkg_id', 'themes', 'locations', 'persons', 'organizations', 'tone', 'positive_score', 'negative_score', 'polarity', 'activity_density', 'selfgroup_density', 'word_count', 'gcam', 'date_event', 'load_date']
Carregado 140102 linhas de aml_silver.sat_event
hub_event colunas: ['event_hk', 'gkg_id', 'date_id', 'load_date']
sat_event colunas: ['event_hk', 'gkg_id', 'themes', 'locations', 'persons', 'organizations', 'tone', 'positive_score', 'negative_score', 'polarity', 'activity_density', 'selfgroup_density', 'word_count', 'gcam', 'date_event', 'load_date']


,event_hk,gkg_id,date_id,load_date
0,000075ad71bb6d7b874da27ea444ba34,20160320053000-449,20160320,2025-11-03 01:50:34.572363+00:00
1,0000a0852140626b267b59880386fcd9,20160510194500-702,20160510,2025-11-03 01:50:34.572363+00:00
2,0000bd6b020d493bccca8849ecca7438,20160506140000-2379,20160506,2025-11-03 01:50:34.572363+00:00
3,0000d0d26face5a4ae8284e95f58b330,20160217070000-615,20160217,2025-11-03 01:50:34.572363+00:00
4,000111bf3ca11704a4017371caec10af,20160219131500-424,20160219,2025-11-03 01:50:34.572363+00:00


,event_hk,gkg_id,themes,locations,persons,organizations,tone,positive_score,negative_score,polarity,activity_density,selfgroup_density,word_count,gcam,date_event,load_date
0,f38bea67bf51ae163be565945bc02800,20160120074500-888,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"4#London, London, City Of, United Kingdom#UK#U...","4#London, London, City Of, United Kingdom#UK#U...",bruno jactel;kirsty witter;erica h boisvert,None,NaN,608.0,NaN,NaN,NaN,NaN,"B Corp,6069;Prior To Southern Energy Managemen...",2016-01-20,2025-11-03 01:57:42.392431+00:00
1,f38d43d0967063211fe2e5cad642822c,20160108151500-579,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"4#London, London, City Of, United Kingdom#UK#U...","4#London, London, City Of, United Kingdom#UK#U...",kirsty witter,None,608.0,NaN,NaN,NaN,NaN,NaN,"Financial Express Holdings,27",2016-01-08,2025-11-03 01:57:42.392431+00:00
2,f38d5d49cb95301ecd1da245a7e2c9d3,20160225113000-1934,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"4#London, London, City Of, United Kingdom#UK#U...","4#London, London, City Of, United Kingdom#UK#U...",kirsty witter,None,608.0,NaN,NaN,NaN,NaN,NaN,"Financial Express Holdings,27",2016-02-25,2025-11-03 01:57:42.392431+00:00
3,f38d93cceb71990a7003f55b75559c5a,20160311094500-1269,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"4#London, London, City Of, United Kingdom#UK#U...","4#London, London, City Of, United Kingdom#UK#U...",kirsty witter,None,608.0,NaN,NaN,NaN,NaN,NaN,"Financial Express Holdings,27",2016-03-11,2025-11-03 01:57:42.392431+00:00
4,f38dfb00b2a97ec7fd4fc6bd6ae3e4ff,20160404060000-26,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,1#Iceland#IC#IC#65#-18#IC;1#Germany#GM#GM#51#9...,1#Russia#RS#RS##60#100#RS#2004;1#Russia#RS#RS#...,anna sigurlaug;suddeutsche zeitung;hosni mubar...,None,NaN,NaN,NaN,NaN,NaN,NaN,"Bank Rossiya,1996;International Consortium Of ...",2016-04-04,2025-11-03 01:57:42.392431+00:00


### Seção 4: Dicionário Básico (Inferido)
Abaixo geramos um resumo automático de tipos e % missing para cada tabela.

In [18]:
# Seção 4: Perfil Inicial de Colunas
def profile(df: pd.DataFrame, name: str):
    if df.empty: return pd.DataFrame(columns=['col','dtype','n_missing','pct_missing'])
    prof = []
    for c in df.columns:
        nmiss = df[c].isna().sum()
        prof.append({'col': c, 'dtype': df[c].dtype, 'n_missing': nmiss, 'pct_missing': nmiss/len(df) if len(df)>0 else 0})
    return pd.DataFrame(prof).sort_values('pct_missing', ascending=False)

profile_hub = profile(hub_event_df, 'hub_event')
profile_sat = profile(sat_event_df, 'sat_event')
display(profile_hub)
display(profile_sat)

,col,dtype,n_missing,pct_missing
0,event_hk,object,0,0.0
1,gkg_id,object,0,0.0
2,date_id,int64,0,0.0
3,load_date,"datetime64[ns, UTC]",0,0.0


,col,dtype,n_missing,pct_missing
6,tone,object,140102,1.000000
12,word_count,float64,132703,0.947188
11,selfgroup_density,float64,131060,0.935461
10,activity_density,float64,129795,0.926432
9,polarity,float64,127305,0.908659
8,negative_score,float64,124841,0.891072
7,positive_score,float64,97628,0.696835
0,event_hk,object,0,0.000000
1,gkg_id,object,0,0.000000
2,themes,object,0,0.000000


In [19]:
# Seção 5: Derivação de País e Data a Partir de sat_event
if 'locations' in sat_event_df.columns:
    sat_event_df['country_codes'] = sat_event_df['locations'].apply(extract_country_codes)
    sat_exploded = sat_event_df.explode('country_codes')
else:
    sat_event_df['country_codes'] = []
    sat_exploded = sat_event_df.copy()
sat_exploded['country_iso3'] = sat_exploded['country_codes'].apply(to_iso3)
sat_exploded = sat_exploded[~sat_exploded['country_iso3'].isna()]
date_col = 'date_event' if 'date_event' in sat_exploded.columns else ('load_date' if 'load_date' in sat_exploded.columns else None)
if date_col:
    sat_exploded['date'] = pd.to_datetime(sat_exploded[date_col]).dt.date
else:
    sat_exploded['date'] = datetime.utcnow().date()  # fallback
print('Linhas após explode:', len(sat_exploded))
display(sat_exploded.head())

Linhas após explode: 293703


,event_hk,gkg_id,themes,locations,persons,organizations,tone,positive_score,negative_score,polarity,activity_density,selfgroup_density,word_count,gcam,date_event,load_date,country_codes,country_iso3,date
4,f38dfb00b2a97ec7fd4fc6bd6ae3e4ff,20160404060000-26,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,1#Iceland#IC#IC#65#-18#IC;1#Germany#GM#GM#51#9...,1#Russia#RS#RS##60#100#RS#2004;1#Russia#RS#RS#...,anna sigurlaug;suddeutsche zeitung;hosni mubar...,None,NaN,NaN,NaN,NaN,NaN,NaN,"Bank Rossiya,1996;International Consortium Of ...",2016-04-04,2025-11-03 01:57:42.392431+00:00,RS,SRB,2016-04-04
4,f38dfb00b2a97ec7fd4fc6bd6ae3e4ff,20160404060000-26,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,1#Iceland#IC#IC#65#-18#IC;1#Germany#GM#GM#51#9...,1#Russia#RS#RS##60#100#RS#2004;1#Russia#RS#RS#...,anna sigurlaug;suddeutsche zeitung;hosni mubar...,None,NaN,NaN,NaN,NaN,NaN,NaN,"Bank Rossiya,1996;International Consortium Of ...",2016-04-04,2025-11-03 01:57:42.392431+00:00,SY,SYR,2016-04-04
4,f38dfb00b2a97ec7fd4fc6bd6ae3e4ff,20160404060000-26,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,1#Iceland#IC#IC#65#-18#IC;1#Germany#GM#GM#51#9...,1#Russia#RS#RS##60#100#RS#2004;1#Russia#RS#RS#...,anna sigurlaug;suddeutsche zeitung;hosni mubar...,None,NaN,NaN,NaN,NaN,NaN,NaN,"Bank Rossiya,1996;International Consortium Of ...",2016-04-04,2025-11-03 01:57:42.392431+00:00,PM,SPM,2016-04-04
4,f38dfb00b2a97ec7fd4fc6bd6ae3e4ff,20160404060000-26,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,1#Iceland#IC#IC#65#-18#IC;1#Germany#GM#GM#51#9...,1#Russia#RS#RS##60#100#RS#2004;1#Russia#RS#RS#...,anna sigurlaug;suddeutsche zeitung;hosni mubar...,None,NaN,NaN,NaN,NaN,NaN,NaN,"Bank Rossiya,1996;International Consortium Of ...",2016-04-04,2025-11-03 01:57:42.392431+00:00,GM,GMB,2016-04-04
4,f38dfb00b2a97ec7fd4fc6bd6ae3e4ff,20160404060000-26,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,1#Iceland#IC#IC#65#-18#IC;1#Germany#GM#GM#51#9...,1#Russia#RS#RS##60#100#RS#2004;1#Russia#RS#RS#...,anna sigurlaug;suddeutsche zeitung;hosni mubar...,None,NaN,NaN,NaN,NaN,NaN,NaN,"Bank Rossiya,1996;International Consortium Of ...",2016-04-04,2025-11-03 01:57:42.392431+00:00,EG,EGY,2016-04-04


In [20]:
# Seção 6: Métricas Agregadas por País/Data (Satélite)
tone_col = 'tone' if 'tone' in sat_exploded.columns else None
polarity_col = 'polarity' if 'polarity' in sat_exploded.columns else None
agg_dict = {'event_cnt': ('event_hk','count')}
if tone_col: agg_dict['tone_mean'] = (tone_col,'mean')
if polarity_col: agg_dict['polarity_mean'] = (polarity_col,'mean')
sat_metrics = sat_exploded.groupby(['country_iso3','date']).agg(**agg_dict).reset_index()
print('sat_metrics shape', sat_metrics.shape)
display(sat_metrics.head())

sat_metrics shape (13400, 5)


,country_iso3,date,event_cnt,tone_mean,polarity_mean
0,AFG,2016-01-01,3,NaN,NaN
1,AFG,2016-01-04,3,NaN,NaN
2,AFG,2016-01-06,5,NaN,NaN
3,AFG,2016-01-08,2,NaN,NaN
4,AFG,2016-01-09,3,NaN,NaN


In [21]:
# Seção 7: Métricas de Hub - Contagem Distinta de event_hk por País/Data via Join
if not hub_event_df.empty and 'event_hk' in hub_event_df.columns and 'event_hk' in sat_exploded.columns:
    hub_with_country = hub_event_df.merge(sat_exploded[['event_hk','country_iso3','date']], on='event_hk', how='left')
    hub_metrics = hub_with_country.groupby(['country_iso3','date']).agg(hub_events=('event_hk','nunique')).reset_index()
else:
    hub_metrics = pd.DataFrame(columns=['country_iso3','date','hub_events'])
print('hub_metrics shape', hub_metrics.shape)
display(hub_metrics.head())

hub_metrics shape (13400, 3)


,country_iso3,date,hub_events
0,AFG,2016-01-01,3
1,AFG,2016-01-04,3
2,AFG,2016-01-06,5
3,AFG,2016-01-08,2
4,AFG,2016-01-09,3


In [22]:
# Seção 8: Base Integrada & Normalizações
base = sat_metrics.merge(hub_metrics, on=['country_iso3','date'], how='left')
base['hub_events'] = base['hub_events'].fillna(0)
if 'tone_mean' in base.columns: base['tone_norm'] = safe_norm(base['tone_mean'].fillna(0))
if 'polarity_mean' in base.columns: base['polarity_norm'] = safe_norm(base['polarity_mean'].fillna(0))
base['event_cnt_norm'] = safe_norm(base['event_cnt'])
base['hub_events_norm'] = safe_norm(base['hub_events'])
# Exemplo de score simples (ajuste conforme necessidade)
base['simple_score'] = (
    base.get('tone_norm', pd.Series([0]*len(base))) * 0.4 +
    base['event_cnt_norm'] * 0.4 +
    base['hub_events_norm'] * 0.2
)
base['simple_score_norm'] = safe_norm(base['simple_score'])
print('Base integrada shape', base.shape)
display(base.head())

Base integrada shape (13400, 12)


C:\Users\gonca\AppData\Local\Temp\ipykernel_28124\610118171.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if 'tone_mean' in base.columns: base['tone_norm'] = safe_norm(base['tone_mean'].fillna(0))


,country_iso3,date,event_cnt,tone_mean,polarity_mean,hub_events,tone_norm,polarity_norm,event_cnt_norm,hub_events_norm,simple_score,simple_score_norm
0,AFG,2016-01-01,3,NaN,NaN,3,0,0.0,0.001358,0.001359,0.000815,0.001358
1,AFG,2016-01-04,3,NaN,NaN,3,0,0.0,0.001358,0.001359,0.000815,0.001358
2,AFG,2016-01-06,5,NaN,NaN,5,0,0.0,0.002716,0.002717,0.001630,0.002716
3,AFG,2016-01-08,2,NaN,NaN,2,0,0.0,0.000679,0.000679,0.000407,0.000679
4,AFG,2016-01-09,3,NaN,NaN,3,0,0.0,0.001358,0.001359,0.000815,0.001358


In [23]:
# Seção 9: Correlações
cols_for_corr = [c for c in ['event_cnt','tone_mean','polarity_mean','hub_events','simple_score_norm'] if c in base.columns]
if len(cols_for_corr) > 1:
    corr_df = base[cols_for_corr].corr()
    print('Matriz de correlação:')
    display(corr_df)
else:
    print('Colunas insuficientes para correlação.')

Matriz de correlação:


,event_cnt,tone_mean,polarity_mean,hub_events,simple_score_norm
event_cnt,1.000000,NaN,-0.055392,1.000000,1.000000
tone_mean,NaN,NaN,NaN,NaN,NaN
polarity_mean,-0.055392,NaN,1.000000,-0.055391,-0.055392
hub_events,1.000000,NaN,-0.055391,1.000000,1.000000
simple_score_norm,1.000000,NaN,-0.055392,1.000000,1.000000


In [24]:
# Seção 10: Rankings de Países
rank_country = base.groupby('country_iso3').agg(
    avg_score=('simple_score_norm','mean'),
    total_events=('event_cnt','sum'),
    avg_tone=('tone_mean','mean') if 'tone_mean' in base.columns else ('event_cnt','count')
).reset_index().sort_values('avg_score', ascending=False)
print('Ranking países (top 15):')
display(rank_country.head(15))

Ranking países (top 15):


,country_iso3,avg_score,total_events,avg_tone
149,USA,0.226713,54254,NaN
129,SPM,0.074458,17042,NaN
26,CHE,0.063916,15411,NaN
65,IND,0.051272,12394,NaN
46,FRA,0.049330,11931,NaN
134,SWZ,0.045920,11049,NaN
130,SRB,0.039834,9666,NaN
55,GMB,0.039465,9578,NaN
90,MEX,0.035018,8464,NaN
20,BRA,0.033495,8153,NaN


In [25]:
# Seção 11: Detecção Simples de Outliers (Z-Score em event_cnt)
if not base.empty:
    base['event_cnt_z'] = (base['event_cnt'] - base['event_cnt'].mean()) / (base['event_cnt'].std() if base['event_cnt'].std() else 1)
    spikes = base[base['event_cnt_z'].abs() >= 3]
    print('Spikes identificados (|z|>=3):', len(spikes))
    display(spikes.head())
else:
    print('Base vazia, sem outliers.')

Spikes identificados (|z|>=3): 255


,country_iso3,date,event_cnt,tone_mean,polarity_mean,hub_events,tone_norm,polarity_norm,event_cnt_norm,hub_events_norm,simple_score,simple_score_norm,event_cnt_z
123,AFG,2016-05-12,240,NaN,894.500000,240,0,0.020189,0.162254,0.162364,0.097374,0.162291,3.070771
333,ARE,2016-01-30,280,NaN,2016.950000,280,0,0.045523,0.189409,0.189538,0.113671,0.189452,3.634004
544,ARG,2016-04-04,263,NaN,1320.666667,263,0,0.029808,0.177868,0.177989,0.106745,0.177909,3.394630
545,ARG,2016-04-05,320,NaN,371.555556,320,0,0.008386,0.216565,0.216712,0.129968,0.216614,4.197237
547,ARG,2016-04-07,306,NaN,507.000000,306,0,0.011443,0.207060,0.207201,0.124264,0.207107,4.000105


In [26]:
# Seção 12: Visualizações
try:
    import plotly.express as px
except ImportError:
    from importlib import reload
    import subprocess, sys as _sys
    subprocess.run([_sys.executable, '-m', 'pip', 'install', 'plotly'], check=False)
    import plotly.express as px
if not base.empty:
    ts_df = base.copy()
    ts_df['date_dt'] = pd.to_datetime(ts_df['date'])
    fig_line = px.line(ts_df.sort_values('date_dt'), x='date_dt', y='simple_score_norm', color='country_iso3', title='Simple Score por País vs Data')
    fig_line.show()
    # Heatmap país x data (pivot score)
    pivot = ts_df.pivot_table(index='country_iso3', columns='date', values='simple_score_norm', aggfunc='mean')
    pivot = pivot.fillna(0)
    fig_heat = px.imshow(pivot.values, labels=dict(x='Data', y='País', color='Score'), x=pivot.columns, y=pivot.index, aspect='auto', title='Heatmap Simple Score')
    fig_heat.show()
    # Scatter correlação (event_cnt vs hub_events)
    if 'hub_events' in ts_df.columns:
        fig_scatter = px.scatter(ts_df, x='event_cnt', y='hub_events', color='country_iso3', title='Event Count vs Hub Events')
        fig_scatter.show()
else:
    print('Sem dados para visualizações.')

In [28]:
# Seção 13: Geração de Insight Textual
def gerar_insights(base_df: pd.DataFrame, rank_df: pd.DataFrame):
    if base_df.empty:
        return 'Sem dados para gerar insights.'
    linhas = []
    linhas.append(f'Total de registros agregados: {len(base_df)}.')
    if not rank_df.empty:
        top = rank_df.head(5)
        top_list = ', '.join(top.country_iso3.tolist())
        linhas.append(f'Top países por score: {top_list}.')
    if 'tone_mean' in base_df.columns:
        linhas.append(f'Média global de tone: {base_df.tone_mean.mean():.2f}.')
    if 'polarity_mean' in base_df.columns:
        linhas.append(f'Média global de polarity: {base_df.polarity_mean.mean():.2f}.')
    if 'hub_events' in base_df.columns:
        linhas.append(f'Média de hub_events por linha agregada: {base_df.hub_events.mean():.2f}.')
    spikes = base_df[base_df.get('event_cnt_z', pd.Series([0]*len(base_df))).abs() >= 3]
    if not spikes.empty:
        linhas.append(f'Foram detectados {len(spikes)} picos (|z|>=3) em event_cnt.')
    return '
'.join(linhas)

insight_text = gerar_insights(base, rank_country)
print('Resumo Insights:
', insight_text)

SyntaxError: EOL while scanning string literal (2379578733.py, line 20)

In [29]:
# Seção 14: Exportação de Resultados (CSV / Parquet)
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
def export_df(df: pd.DataFrame, nome: str):
    if df.empty:
        print('Nada para exportar em', nome)
        return
    csv_path = f'{nome}_{timestamp}.csv'
    parquet_path = f'{nome}_{timestamp}.parquet'
    df.to_csv(csv_path, index=False)
    try:
        df.to_parquet(parquet_path, index=False)
    except Exception as e:
        print('Falha ao exportar parquet:', e)
    print('Exportado:', csv_path, parquet_path)
export_df(base, 'base_integrada')
export_df(rank_country, 'ranking_paises')

Exportado: base_integrada_20251110_193748.csv base_integrada_20251110_193748.parquet
Exportado: ranking_paises_20251110_193748.csv ranking_paises_20251110_193748.parquet


### Seção 15: Próximos Passos
- Adicionar pesos customizados para composição do score.
- Incluir filtragem por janela temporal (últimos N dias).
- Persistir agregações em nova tabela analytics no Postgres.
- Criar testes de integridade (not_null, unique) para colunas-chave.
- Refinar detecção de anomalias (EWMA, STL, Prophet, etc.).

> Ajuste o notebook conforme sua evolução de métricas e governança de dados.